In [1]:
import numpy as np
import pandas as pd

import scvi
from scvi.dataset.pbmc import PbmcDataset
from scvi.dataset.loom import RetinaDataset

import anndata

In [2]:
# create a python script from the notebook
!jupyter nbconvert generate_data.ipynb --to script

[NbConvertApp] Converting notebook generate_data.ipynb to script
[NbConvertApp] Writing 4498 bytes to generate_data.py


## Configurations

In [3]:
# this sets the file name prefix for generated files
prefix = "retina_eq_ct"

# choose a dataset to use
#gene_dataset = PbmcDataset()
gene_dataset = RetinaDataset()

# optionally, balance the cell-type composition between the batches
equal_cell_types = False

File data/retina.loom already downloaded
Preprocessing dataset
Finished preprocessing dataset


## Get cell-type and batch labels from scVI dataset object
For each dataset we need: 
    (1) an gene expression count matrix 
    (2) cell type labels
    (3) batch labels

In [4]:
# get cell type labels
cell_types = [gene_dataset.cell_types[i] for i in gene_dataset.labels.ravel()]

# labels = pd.DataFrame(gene_dataset.batch_indices, columns=["batch_cov"])
# labels["ct_cov"] = cell_types
# labels.to_csv(prefix + "_batch_labels.csv", sep=",", header=True, index=False)

In [5]:
# batch labels
gene_dataset.batch_indices

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [6]:
print(len(gene_dataset.batch_indices))

19829


## Get the expression data, batch labels, and cell type labels into an AnnData object

In [7]:
ad = anndata.AnnData(gene_dataset.X, obs={"batch_cov": list([i[0] for i in gene_dataset.batch_indices]), "ct_cov": cell_types})

In [8]:
print(ad.X.shape)

(19829, 13166)


In [9]:
# for each batch, look at distribution of cell types
for batch in np.unique(ad.obs["batch_cov"]):
    print("="*30)
    print(batch)
    print("="*30)
    print(ad.obs[ad.obs["batch_cov"]==batch]["ct_cov"].value_counts())
    print(ad.obs[ad.obs["batch_cov"]==batch]["ct_cov"].value_counts()/ad.obs[ad.obs["batch_cov"]==batch].shape[0])

0
RBC      5310
MG        920
BC5A      812
BC6       641
BC7       614
BC1A      381
BC5C      367
BC3B      264
BC1B      252
BC3A      190
BC2       155
BC5D      112
BC4        90
BC8_9      73
BC5B       53
Name: ct_cov, dtype: int64
RBC      0.518859
MG       0.089896
BC5A     0.079343
BC6      0.062634
BC7      0.059996
BC1A     0.037229
BC5C     0.035861
BC3B     0.025796
BC1B     0.024624
BC3A     0.018566
BC2      0.015146
BC5D     0.010944
BC4      0.008794
BC8_9    0.007133
BC5B     0.005179
Name: ct_cov, dtype: float64
1
RBC      2865
MG       1295
BC5A      871
BC7       698
BC5C      667
BC6       622
BC1A      454
BC3B      348
BC1B      333
BC5B      299
BC5D      290
BC2       265
BC3A      230
BC4       214
BC8_9     144
Name: ct_cov, dtype: int64
RBC      0.298593
MG       0.134966
BC5A     0.090776
BC7      0.072746
BC5C     0.069515
BC6      0.064825
BC1A     0.047316
BC3B     0.036269
BC1B     0.034706
BC5B     0.031162
BC5D     0.030224
BC2      0.027619
BC3A   

## Optional: subsample cells so that cell type compositions match between batches


In [10]:
if equal_cell_types:
    # subsample the cells so that cell type proportions match in each batch
    min_num_cells_per_cell_type = {}
    for ct in ad.obs["ct_cov"].unique():
        min_num_cells = np.min(ad.obs[ad.obs["ct_cov"]==ct]["batch_cov"].value_counts())
        min_num_cells_per_cell_type[ct] = min_num_cells
    print(min_num_cells_per_cell_type)

    print(ad.shape)

    # for a batch, if we find it has more than the minimum number of a particular cell type, downsample
    final_indices = []
    for batch in ad.obs["batch_cov"].unique():
        for ct in ad.obs["ct_cov"].unique():
    #        if sc_data[(sc_data.obs["batch_cov"]==batch) & (sc_data.obs["ct_cov"]==ct)].shape[0] > min_num_cells_per_cell_type[ct]:
            indices = ad[(ad.obs["batch_cov"]==batch) & (ad.obs["ct_cov"]==ct)].to_df().index
            indices_to_keep = np.random.choice(indices, size=min_num_cells_per_cell_type[ct], replace=False)
            final_indices = final_indices + list(indices_to_keep)

    print(len(final_indices))
    ad = ad[final_indices, :]
    print(ad.shape)
    
    # for each batch, look at distribution of cell types
    for batch in ad.obs["batch_cov"].unique():
        print("="*30)
        print(batch)
        print("="*30)
        print(ad.obs[ad.obs["batch_cov"]==batch]["ct_cov"].value_counts())
        print(ad.obs[ad.obs["batch_cov"]==batch]["ct_cov"].value_counts()/ad.obs[ad.obs["batch_cov"]==batch].shape[0])

In [11]:
# create the cells x genes matrix csv files
try: 
    pd.DataFrame(ad.X.todense(), columns=ad.var_names).to_csv(prefix + ".csv", sep=",", header=True, index=False)
except AttributeError:
    pd.DataFrame(ad.X, columns=ad.var_names).to_csv(prefix + ".csv", sep=",", header=True, index=False)

In [12]:
# this step is for generating data for DCA (requires genes x cells matrix)
try: 
    pd.DataFrame(ad.X.todense(), columns=ad.var_names).T.to_csv(prefix + "_T.csv", sep=",", header=True, index=False)
except AttributeError:
    pd.DataFrame(ad.X, columns=ad.var_names).T.to_csv(prefix + "_T.csv", sep=",", header=True, index=False)

In [13]:
# get cell type labels from object
cell_types = ad.obs["ct_cov"]

#labels = pd.DataFrame(gene_dataset.batch_indices, columns=["batch_cov"])
labels = pd.DataFrame(ad.obs["batch_cov"], columns=["batch_cov"])
labels["ct_cov"] = cell_types
print(labels.head(20))
print(labels.shape)
labels.to_csv(prefix + "_batch_labels.csv", sep=",", header=True, index=False)

    batch_cov ct_cov
0           0   BC5A
1           0   BC5C
2           0    BC6
3           0    RBC
4           1   BC1A
5           0    RBC
6           1    BC7
7           1   BC5A
8           0    RBC
9           0    RBC
10          1   BC1A
11          1   BC5A
12          0    RBC
13          1   BC5D
14          0    RBC
15          1   BC1A
16          1   BC3A
17          0    RBC
18          0   BC5D
19          1    RBC
(19829, 2)


In [14]:
# write AnnDataset to h5ad file
ad.write(prefix + ".h5ad")

... storing 'ct_cov' as categorical
